# Part 1

In [1]:
def split_list(input_list):
    sub_list = []
    final_list = []
    for item in input_list:
        if item == '':
            final_list.append(sub_list)
            sub_list = []
        else:
            sub_list.append(item)
    if sub_list:
        final_list.append(sub_list)
    return final_list


In [2]:
class Map:
    def __init__(self, map: list[str]):
        self.destination_range_start, self.source_range_start, self.range_length = [int(i) for i in map.split(" ")]
        self.diff = self.destination_range_start - self.source_range_start
        self.dest_range = (self.destination_range_start, self.destination_range_start + self.range_length) 
        self.source_range = (self.source_range_start, self.source_range_start + self.range_length) 


class Mapper:
    def __init__(self, line: str):
        self.map_name = line[0]
        self.map_from, self.map_to = self.map_name.split(" ")[0].split("-to-")
        self.ranges = line[1:]
        self.maps = list()

        for map in self.ranges:
            self.maps.append(Map(map))
        self.maps = sorted(self.maps, key=lambda x: x.source_range_start)

    def map(self, i):
        for map in self.maps:
            if map.source_range_start <= i < map.source_range_start + map.range_length:
                return i + map.diff
        else:
            return i

In [3]:
with open("inputs/day5.txt") as fn:
    lines = fn.read().splitlines()
    instructions = split_list(lines)

In [4]:
def part1(instructions):
    seeds = instructions[0]
    seeds = [int(i) for i in seeds[0].split(": ")[1].split(" ")]
    maps = [Mapper(m) for m in instructions[1:]]

    locations = list()
    for seed in seeds:
        num = seed
        for map in maps:
            num = map.map(num)
        locations.append(num)
    return min(locations)

In [5]:
part1(instructions)

84470622

# Part 2

Plus extended the class in part 1 to work on ranges

In [6]:
class MapperPart2(Mapper):        
    def map_range(self, range):
        ranges_out = []
        
        for map in self.maps:
            if range[1] < map.source_range[0]: # entirely below
                continue

            if range[0] > map.source_range[1]: # entirely above
                continue

            if range[0] < map.source_range[0]: # start missed
                ranges_out.append((range[0], map.source_range[0]))
                range = (map.source_range[0], range[1])

            if range[1] <= map.source_range[1]: # wholey contained
                ranges_out.append((range[0] + map.diff, range[1] + map.diff))
                break
            else:
                ranges_out.append((range[0] + map.diff, map.dest_range[1]))
                range = (map.source_range[1], range[1])
        else:
            ranges_out.append((range[0], range[1]))

        return ranges_out

In [7]:
def part2(instructions):
    seeds = instructions[0]
    seeds = [int(i) for i in seeds[0].split(": ")[1].split(" ")]
    ranges = [(seeds[i*2],seeds[i*2] + seeds[(i*2)+1]) for i in range(int(len(seeds)/2))]

    for instr in instructions[1:]:
        ranges = sum([MapperPart2(instr).map_range(range) for range in ranges], [])

    print(min((r[0] for r in ranges)))

part2(instructions)

26714516
